# Push Table Tags

This notebook provides an example for pushing table tags from Secoda to BigQuery. To get started, you will need to [obtain an API key](https://app.secoda.co/settings/api)

Input your API key and Secoda API endpoint URL below. The URL for the cloud instance is `https://api.secoda.co`. If you are self-hosting Secoda or on the EU instance, you will have to update variable.

In [ ]:
%pip install google google-api-core google-api-python-client google-auth google-auth-httplib2 googleapis-common-protos httplib2

In [16]:

import requests
import httplib2
import google_auth_httplib2
from google.oauth2 import service_account
from googleapiclient.discovery import build

API_KEY = "api_key"
SECODA_API_URL = "https://api.secoda.co"
BIGQUERY_PROJECT_ID = "project_id"
BIGQUERY_JSON_CREDENTIALS = {
  "type": "service_account",
  "project_id": "project_id",
  "private_key_id": "",
  "private_key": "",
  "client_email": "account@secoda-web.iam.gserviceaccount.com",
  "client_id": "",
  "auth_uri": "https://accounts.google.com/o/oauth2/auth",
  "token_uri": "https://oauth2.googleapis.com/token",
  "auth_provider_x509_cert_url": "https://www.googleapis.com/oauth2/v1/certs",
  "client_x509_cert_url": "https://www.googleapis.com/robot/v1/metadata/x509/account%40project_id.iam.gserviceaccount.com"
}

session = requests.Session()
session.headers.update(dict(
    Authorization=f"Bearer {API_KEY}"
))

class BigQueryHelpers:
    DATE_LENGTH = 8
    DEFAULT_PAGE_SIZE = 300
    NUM_RETRIES = 1

    DEFAULT_SCOPES = ["https://www.googleapis.com/auth/cloud-platform"]

    @staticmethod
    def _is_sharded_table(table_id: str) -> bool:
        suffix = table_id[-BigQueryHelpers.DATE_LENGTH :]
        return suffix.isdigit()

http = httplib2.Http()
authed_http = google_auth_httplib2.AuthorizedHttp(
    # we have to re-create the credentials here instead of using the self.credentials
    # call due to the different scopes these credentials require
    service_account.Credentials.from_service_account_info(
        BIGQUERY_JSON_CREDENTIALS,
        scopes=BigQueryHelpers.DEFAULT_SCOPES,
    ),
    http=http,
)
bigquery_service = build(
    "bigquery", "v2", http=authed_http, cache_discovery=False
)


def build_url(url: str):
    return f"{SECODA_API_URL}{url}"

## Get Table Tags

You can obtain a table's tags with the table's ID or with the name, schema, and database. We'll use the name, schema, and database example

In [ ]:
def get_tag(id):
    res = session.get(build_url(f"/tag/{id}/"))
    print(res.text)
    return res.json()

def get_table(title, schema, database, *args, **kwargs):
    res = session.get(
        build_url(f"/table/tables/?title={title}&parent__title={schema}&parent__parent__title={database}")
    )
    results = res.json().get("results")
    if len(results) >= 1:
        return results[0]
    return None

table = {
    "title": "customers",
    "schema": "dbt_amcewen",
    "database": "secoda-web",
}

table = get_table(**table)
tags = table.get("tags", [])

for tag_id in tags:
    tag = get_tag(tag_id)
    bigquery_service.tables().patch(
        projectId=BIGQUERY_PROJECT_ID,
        datasetId=table.get("schema"),
        tableId=table.get("title"),
        body={
            "labels": {
                tag.get("name").lower(): tag.get("name").lower()
            }
        }
    ).execute()


